<a href="https://colab.research.google.com/github/yinghuing/K_drama_recommender/blob/master/chapter2/openai_api_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 0: ハンズオンの準備
---

## API キーの設定
*  左ナビゲーションで [**シークレット**] アイコン (鍵形のアイコン) をクリックします。
*  [**新しいシークレットを追加**] をクリックし、[**名前**] に `OPENAI_API_KEY` と入力し、その [**値**] に指定されたキーを入力します。
*  設定したシークレットの [**ノートブックからのアクセス**] を有効にします。
*  入力が完了したら、下のセルを実行します。

In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Section 1: トークン
---

## Tokenizerとtiktoken
- LLM などの自然言語処理を行う機械学習モデルは、テキストをトークンという単位で処理する。
- テキストデータなどをトークン化 (tokenize) するプログラムを **Tokenizer** という。
- トークン化の仕方や入力可能なトークン数は LLM によって異なり、入力するトークン数に応じて API 利用料金が課金される。
- **tiktoken** は OSS の高速な Tokenizer である。

In [2]:
# !pip -q install tiktoken

In [3]:
!pip -q install tiktoken==0.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:00


In [4]:
!pip freeze | grep tiktoken

tiktoken==0.8.0


In [7]:
!pip freeze

absl-py==1.4.0
accelerate==1.3.0
aiohappyeyeballs==2.6.1
aiohttp==3.11.13
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.5
ale-py==0.10.2
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.1
arviz==0.20.0
astropy==7.0.1
astropy-iers-data==0.2025.3.10.0.29.26
astunparse==1.6.3
atpublic==4.1.0
attrs==25.3.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
beautifulsoup4==4.13.3
betterproto==2.0.0b6
bigframes==1.38.0
bigquery-magics==0.8.0
bleach==6.2.0
blinker==1.9.0
blis==1.2.0
blosc2==3.2.0
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.89
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.21.0
cloudpickle==3.1.1
cmake==3.31.6
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.

In [5]:
import tiktoken

text = "It’s easy to make something cool with LLMs, but very hard to make something production-ready with them."

encoding = tiktoken.encoding_for_model("gpt-4o-mini")
tokens = encoding.encode(text)
print(len(tokens))

23


In [6]:
text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o-mini")
tokens = encoding.encode(text)
print(len(tokens))

37


# Section 2: OpenAI API の利用
---

### プロンプト設計のためのサイト
- [OpenAI Platform](https://platform.openai.com/playground/chat?models=gpt-4o-mini)
- [Google AI Studio](https://aistudio.google.com/)
- [Anthropic Console](https://console.anthropic.com/)

## Chat Completions API

In [ ]:
# !pip install openai

In [ ]:
# !pip -q install openai==1.54.4

In [8]:
!pip freeze | grep openai

openai==1.61.1


### Chat Completions APIの呼び出し

In [9]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! I'm John."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello, John! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])


In [10]:
completion.choices[0].message.content

'Hello, John! How can I assist you today?'

### 会話履歴を踏まえた応答を得る

In [11]:
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."},
        {"role": "assistant", "content": "Hello John! How can I assist you today?"},
        {"role": "user", "content": "Do you know my name?"}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Yes, you mentioned that your name is John. How can I help you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])


### ストリーミングで応答を得る

In [12]:
stream = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
        {"role": "system", "content": "あなたは AI や機械学習に精通したプロフェッショナルです。"},
        {"role": "user", "content": "AI とは何ですか。"}
  ],
  stream=True
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

AI（人工知能）とは、人間の知能を模倣することを目指したコンピュータシステムやプログラムのことを指します。AIは、さまざまな技術や手法を用いて、データの解析、パターン認識、意思決定、自然言語処理、画像認識などのタスクを実行します。

AIは大きく分けて二つのカテゴリに分類されます：

1. **狭義のAI（Weak AI）**: 特定のタスクを実行することに特化しているAIです。例えば、音声アシスタントや画像認識ソフトウェアがこれに含まれます。

2. **汎用AI（Strong AI）**: 人間のように広範な知識や能力を持ち、さまざまなタスクを自律的に遂行できるAIを指します。現在のところ、汎用AIは理論的な概念に留まっており、実用化されていません。

AI技術には、機械学習（ML）、深層学習（DL）、自然言語処理（NLP）などが含まれ、それぞれ特定のアプローチや手法を用いています。AIは医療、金融、製造業、エンターテインメントなど、さまざまな分野で活用され、効率性の向上や新しいビジネスチャンスの創出に寄与しています。

# Section 3: Function calling
---
Function Calling に対応しているモデルでは、利用可能な関数の情報をモデルに渡すことで、モデルに必要に応じた関数の利用を選択させることができる。

(※ 以下では [OpenAI の公式ドキュメント](https://platform.openai.com/docs/guides/function-calling) をもとに一部改変したコードを使用している)

## 関数を用意する  
天気予報の情報を出力する関数  
(実際に外部の天気予報サービスを利用するのではなく、既定の予報を返す疑似的な天気予報の関数)

In [13]:
import json

def get_current_weather(location, unit="celsius"):
    weather_info = {
        "location": location,
        "temperature": "25",
        "unit": "celsius",
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

## 関数のリストを定義する
- 利用可能な関数のリストを定義する。  
- それぞれの関数については、辞書で関数名や関数についての説明、関数を呼び出す際のパラメータ等を定義する。  
- 下の例では、`tools` という名前のリストに要素として関数が 1 つだけ入っている。

In [14]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Tokyo",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

## 関数のリストを渡して LLM を呼び出す

In [15]:
messages = [{"role": "user", "content": "What's the weather like in Tokyo?"}]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

print(response.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8F6D8qQwyGaAxQj9AiK2pbeq', function=Function(arguments='{"location":"Tokyo"}', name='get_current_weather'), type='function')], annotations=[])


In [16]:
print(response.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_8F6D8qQwyGaAxQj9AiK2pbeq', function=Function(arguments='{"location":"Tokyo"}', name='get_current_weather'), type='function')]


In [17]:
print(response.choices[0].message.tool_calls[0].function.name)
print(response.choices[0].message.tool_calls[0].function.arguments)

get_current_weather
{"location":"Tokyo"}


- LLM が関数を使用することを選択した場合、使用する関数の名前や関数を呼び出す際のパラメータを回答する。
- LLM は `tools` で定義されている関数の `name` や `description` 、`parameters` を参照して回答を生成する。

## 関数を実行する  
- 上記の LLM の回答から関数名やパラメータを取得し、関数を実行する

In [18]:
response_message = response.choices[0].message
tool_call = response_message.tool_calls[0]

available_functions = {
    "get_current_weather": get_current_weather,
}
messages.append(response_message)
function_name = tool_call.function.name
function_to_call = available_functions[function_name]
function_args = json.loads(tool_call.function.arguments)
function_response = function_to_call(
    location=function_args.get("location"),
    unit=function_args.get("unit"),
)

print(function_response)

{"location": "Tokyo", "temperature": "25", "unit": "celsius", "forecast": ["sunny", "windy"]}


## 関数の実行結果を `messages` に追加する

In [19]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": function_response,
})

## `messages` を渡して LLM を再度呼び出す

In [20]:
second_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

print(second_response.choices[0].message)

ChatCompletionMessage(content='The current weather in Tokyo is 25°C, sunny, and windy.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])
